In [1]:
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor
import os
import random
from urllib.error import HTTPError

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [2]:
def save_to_csv(df, filename):
    if not os.path.isfile(filename):
        df.to_csv(filename, index=False)
    else:  
        df.to_csv(filename, mode='a', header=False, index=False)

In [3]:
names_df = pd.read_csv(r"C:\Users\gabri\Documents\nhl_players_dataset.csv")

In [4]:
names_df.head()

,Player Names,Player Links,Position
0,Antti Aalto,/players/a/aaltoan01.html,C
1,George Abbott,/players/a/abbotge01.html,G
2,Reg Abbott,/players/a/abbotre01.html,C
3,Spencer Abbott,/players/a/abbotsp01.html,LW
4,Justin Abdelkader,/players/a/abdelju01.html,LW


In [5]:
df_goalies = names_df[names_df['Position'] == 'G']
df_goalies.head()

,Player Names,Player Links,Position
1,George Abbott,/players/a/abbotge01.html,G
28,John Adams,/players/a/adamsjo02.html,G
33,David Aebischer,/players/a/aebisda01.html,G
48,John Aiken,/players/a/aikendo01.html,G
52,Andy Aitkenhead,/players/a/aitkean01.html,G


In [6]:
df_players = names_df[names_df['Position'] != 'G']
df_players.head()

,Player Names,Player Links,Position
0,Antti Aalto,/players/a/aaltoan01.html,C
2,Reg Abbott,/players/a/abbotre01.html,C
3,Spencer Abbott,/players/a/abbotsp01.html,LW
4,Justin Abdelkader,/players/a/abdelju01.html,LW
5,Clarence Abel,/players/a/abelcl01.html,D


In [7]:
goalie_name_list = df_goalies['Player Names'].tolist()
goalie_link_list = df_goalies['Player Links'].tolist()

player_name_list = df_players['Player Names'].tolist()
player_link_list = df_players['Player Links'].tolist()

In [8]:
'''
full_df_goalies = pd.DataFrame()

def scrape_stats_goalies(player):

    url = f"https://www.hockey-reference.com{player}"
    full_df = pd.read_html(url)
    goalie_df = full_df[0]
    full_df_goalies = pd.concat([full_df_goalies, goalie_df], ignore_index=True)
    time.sleep(random.randint(10, 20))

with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(scrape_stats_goalies, goalie_link_list)
'''

'\nfull_df_goalies = pd.DataFrame()\n\ndef scrape_stats_goalies(player):\n\n    url = f"https://www.hockey-reference.com{player}"\n    full_df = pd.read_html(url)\n    goalie_df = full_df[0]\n    full_df_goalies = pd.concat([full_df_goalies, goalie_df], ignore_index=True)\n    time.sleep(random.randint(10, 20))\n\nwith ThreadPoolExecutor(max_workers=5) as executor:\n    results = executor.map(scrape_stats_goalies, goalie_link_list)\n'

In [9]:
#def scrape_stats_goalies(player):
#    try:
#        url = f"https://www.hockey-reference.com{player}"
#        full_df = pd.read_html(url)
#        goalie_df = full_df[0]
#        time.sleep(random.randint(35, 45))  # Simulate delay
#        return goalie_df  # Return the DataFrame for this player
#    except Exception as e:
#        print(f"Error scraping {player}: {e}")
#        return pd.DataFrame()  # Return an empty DataFrame in case of an error
#
#with ThreadPoolExecutor(max_workers=5) as executor:
#    results = executor.map(scrape_stats_goalies, goalie_link_list)
#
## Collect all results and concatenate them
#full_df_goalies = pd.concat(results, ignore_index=True)

In [10]:
def scrape_stats_goalies(player, retries=3):
    try:
        url = f"https://www.hockey-reference.com{player}"
        full_df = pd.read_html(url)
        goalie_df = full_df[0]
        time.sleep(random.randint(35, 45))  # Increased delay
        return goalie_df  # Return the DataFrame for this player
    except HTTPError as e:
        if retries > 0:
            print(f"Error scraping {player}: {e}. Retrying...")
            time.sleep(random.randint(35, 45))  # Wait before retrying
            return scrape_stats_goalies(player, retries=retries-1)
        else:
            print(f"Error scraping {player}: {e}. No more retries left.")
            return pd.DataFrame()  # Return an empty DataFrame after retries are exhausted
    except Exception as e:
        print(f"Error scraping {player}: {e}")
        return pd.DataFrame()

with ThreadPoolExecutor(max_workers=3) as executor:  # Reduced number of workers
    results = executor.map(scrape_stats_goalies, goalie_link_list)

# Collect all results and concatenate them
full_df_goalies = pd.concat(results, ignore_index=True)

In [ ]:
full_df_goalies.head(30)

: 

In [ ]:
#df = df.dropna(subset=['Unnamed: 1_level_0',    'Age'])

: 